In [1]:
import math
import faiss

from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS

from langchain.chat_models import ChatOpenAI

from discussion_agents.memory.generative_agents import GenerativeAgentMemory
from discussion_agents.core.base import BaseCore

In [2]:
import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def relevance_score_fn(score: float) -> float:
    return 1.0 - score / math.sqrt(2)

def create_new_memory_retriever():
    embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

LLM = ChatOpenAI(openai_api_key=openai_api_key, max_tokens=1500)

core = BaseCore(
    llm=LLM,
    retriever=create_new_memory_retriever(),
)
memory = GenerativeAgentMemory(
    core=core,
)

In [4]:
instruction = "Describe what makes a table reliable."
name = "Bob"
age = 25
traits = "talkative, enthusiastic"
lifestyle = "lazy, likes to sleep late"
status = ""
_summary = "Bob is a socialite with lots of friends and a heart for enthusiasm and extrovertedness."

summary = (
    f"Name: {name}\n"
    + f"Age: {age}\n"
    + f"Innate traits: {traits}\n"
    + f"Status: {status}\n"
    + f"Lifestyle: {lifestyle}\n"
    + f"{_summary}\n"
)

llm_kwargs = {}

In [5]:
from discussion_agents.planning.generative_agents import (
    generate_broad_plan,
    update_status,
    generate_refined_plan
)

In [6]:
broad_plan = generate_broad_plan(
    instruction=instruction,
    summary=summary,
    core=core
)
broad_plan

['Identify the purpose and intended use of the table',
 'Determine the source and credibility of the data used to populate the table',
 'Ensure that the table is organized and structured in a logical manner',
 'Verify that the table includes all necessary and relevant information',
 'Check for accuracy and consistency of the data within the table',
 'Validate that the table is up to date and reflects the most recent information available',
 'Consider the context and audience of the table to ensure it is easily understandable and user-friendly',
 'Review the table for any potential errors or inconsistencies',
 'Test the functionality of any formulas or calculations within the table',
 'Seek feedback from users or stakeholders to assess the reliability and usefulness of the table']

In [7]:
update_status(
    instruction=instruction,
    previous_steps=broad_plan[:1],
    plan_step=broad_plan[1],
    summary=summary,
    status=status,
    core=core
)

'Status: Bob is feeling confident and motivated to conduct thorough research and critically evaluate sources in order to ensure the reliability of the table. He understands the importance of gathering accurate and trustworthy information and is determined to select sources that align with the goals of the table. With his enthusiasm and talkative nature, Bob is excited to embark on this journey of creating a reliable table.'

In [8]:
refined_steps = generate_refined_plan(
    instruction=instruction,
    previous_steps=broad_plan[:1],
    plan_step=broad_plan[1],
    summary=summary,
    core=core
)
refined_steps

'1) Evaluate the source of the data used to populate the table, ensuring it comes from a reputable and trustworthy source.\n2) Verify the accuracy and reliability of the data by cross-checking it with other reliable sources or conducting independent research.\n3) Consider the methodology used to collect and analyze the data, ensuring it is rigorous and follows appropriate standards.\n4) Assess the completeness and comprehensiveness of the data, ensuring it covers all relevant aspects and provides a comprehensive view.\n5) Consider the timeliness of the data, ensuring it is up-to-date and reflects the most current information available.\n6) Evaluate any potential biases or conflicts of interest that may exist in the data or its source, ensuring it is unbiased and objective.\n7) Consider the relevance and applicability of the data to the purpose and intended use of the table, ensuring it aligns with the specific needs and requirements.'

In [10]:
print(refined_steps)

1) Evaluate the source of the data used to populate the table, ensuring it comes from a reputable and trustworthy source.
2) Verify the accuracy and reliability of the data by cross-checking it with other reliable sources or conducting independent research.
3) Consider the methodology used to collect and analyze the data, ensuring it is rigorous and follows appropriate standards.
4) Assess the completeness and comprehensiveness of the data, ensuring it covers all relevant aspects and provides a comprehensive view.
5) Consider the timeliness of the data, ensuring it is up-to-date and reflects the most current information available.
6) Evaluate any potential biases or conflicts of interest that may exist in the data or its source, ensuring it is unbiased and objective.
7) Consider the relevance and applicability of the data to the purpose and intended use of the table, ensuring it aligns with the specific needs and requirements.
